<a href="https://colab.research.google.com/github/ufbfung/mortgage-calculator/blob/main/Mortgage_Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import streamlit as st
import requests
import json

import matplotlib.pyplot as plt
import numpy as np
from datetime import date, timedelta

def calculate_equity_over_time(total_cost, down_payment_percent, mortgage_interest_rate, mortgage_term_years):
    down_payment = total_cost * down_payment_percent
    loan_amount = total_cost - down_payment
    monthly_payment = (loan_amount * mortgage_interest_rate / 12) / (1 - (1 + mortgage_interest_rate / 12) ** (-mortgage_term_years * 12))
    
    equity = []
    months = []
    total_paid = 0
    remaining_balance = loan_amount
    
    while remaining_balance > 0:
        months.append(len(equity))
        equity_percent = (total_cost - remaining_balance) / total_cost * 100
        equity.append(equity_percent)
        
        interest_payment = remaining_balance * mortgage_interest_rate / 12
        principal_payment = monthly_payment - interest_payment
        total_paid += monthly_payment
        remaining_balance -= principal_payment
    
    plt.plot(months, equity)
    plt.xlabel('Months')
    plt.ylabel('Equity Percentage')
    
    # Add markers for 25%, 50%, and 75% equity
    plt.axhline(y=25, color='gray', linestyle='--')
    plt.axhline(y=50, color='gray', linestyle='--')
    plt.axhline(y=75, color='gray', linestyle='--')
    
    plt.show()
    
    today = date.today()
    
    for i, eq in enumerate(equity):
        if eq >= 25:
            dt = today + timedelta(days=(months[i] * 30))
            years = int((dt - today).days / 365.25)
            print(f"You will have 25% equity on {dt.strftime('%Y-%m-%d')} ({years} years from today).")
            break
    else:
        print('You will not have 25% equity.')
        
    for i, eq in enumerate(equity):
        if eq >= 50:
            dt = today + timedelta(days=(months[i] * 30))
            years = int((dt - today).days / 365.25)
            print(f"You will have 50% equity on {dt.strftime('%Y-%m-%d')} ({years} years from today).")
            break
    else:
        print('You will not have 50% equity.')
        
    for i, eq in enumerate(equity):
        if eq >= 75:
            dt = today + timedelta(days=(months[i] * 30))
            years = int((dt - today).days / 365.25)
            print(f"You will have 75% equity on {dt.strftime('%Y-%m-%d')} ({years} years from today).")
            break
    else:
        print('You will not have 75% equity.')

def main():
    # Define FRED API key and base URL
    fred_api_key = 'ac53cf8d2bfd2dec1edc849ca32d3cbd'
    fred_base_url = 'https://api.stlouisfed.org/fred/'

    # Define the series ID for the most recent 30-year fixed-rate mortgage
    fred_series_id = 'MORTGAGE30US'

    # Define the API request URL
    fred_url = f'{fred_base_url}series/observations?series_id={fred_series_id}&api_key={fred_api_key}&file_type=json&limit=1&sort_order=desc'

    # Send the API request and parse the response JSON
    response = requests.get(fred_url)
    data = json.loads(response.content.decode())

    # Extract the most recent observation and the interest rate
    latest_observation = data['observations'][0]
    interest_rate = float(latest_observation['value'])

    # Set loan term to 30 years
    loan_term_years = 30

    # Get user input for mortgage amount and down payment as a percentage of the mortgage amount
    mortgage_amount = st.number_input('Enter the mortgage amount:', value=400000)
    down_payment_percent = st.slider('Enter the down payment as a percentage of the mortgage amount:', 0, 100, 20)

    # Convert down payment percentage to dollar amount
    down_payment = mortgage_amount * (down_payment_percent / 100)

    # Calculate the loan amount, number of monthly payments, and monthly interest rate
    loan_amount = mortgage_amount - down_payment
    loan_term_months = loan_term_years * 12
    monthly_interest_rate = (interest_rate / 100) / 12

    # Calculate the monthly mortgage payment
    monthly_payment = (loan_amount * monthly_interest_rate) / (1 - (1 + monthly_interest_rate) ** (-loan_term_months))

    # Calculate the estimated property taxes (annual rate is assumed to be 1.2% of the home value)
    property_taxes = mortgage_amount * 0.012 / 12

    # Calculate the estimated home insurance (annual rate is assumed to be 0.4% of the home value)
    home_insurance = mortgage_amount * 0.004 / 12

    # Get user input for HOA fees
    hoa_fees = st.number_input('Enter the monthly HOA fees (if applicable):', value=0)

    # Calculate the estimated mortgage insurance (if applicable)
    if down_payment_percent < 20:
        mortgage_insurance_rate = 0.0085
        mortgage_insurance = (mortgage_amount * mortgage_insurance_rate / 12)
    else:
        mortgage_insurance = 0

    # Calculate the total monthly cost
    total_monthly_cost = monthly_payment + property_taxes + home_insurance + hoa_fees + mortgage_insurance

    # Print the monthly mortgage payment, estimated expenses, and total monthly cost
    st.write(f'The monthly mortgage payment is ${monthly_payment:.2f}')
    st.write(f'Estimated monthly property taxes: ${property_taxes:.2f}')
    st.write(f'Estimated monthly home insurance: ${home_insurance:.2f}')
    st.write(f'Monthly HOA fees: ${hoa_fees:.2f}')
    if down_payment_percent < 20:
        st.write(f'Estimated monthly mortgage insurance: ${mortgage_insurance:.2f}')
    st.write(f'The total monthly cost is ${total_monthly_cost:.2f}')

    # Draw the equity graph
    st.write(f'Equity graph')

if __name__ == '__main__':
    st.set_page_config(page_title='Mortgage Calculator')
    main()

In [ ]:
import streamlit as st
st.set_option('deprecation.showPyplotGlobalUse', False)
import matplotlib.pyplot as plt
import numpy as np
from datetime import date, timedelta

st.sidebar.header('Mortgage Calculator')

total_cost = st.sidebar.slider('Total cost of the property', 100000, 1000000, 500000, 10000)
down_payment_percent = st.sidebar.slider('Down payment percentage', 1, 100, 20)

mortgage_interest_rate = st.sidebar.slider('Mortgage interest rate', 0.0, 10.0, 3.5, step=0.1)
mortgage_term_years = st.sidebar.slider('Mortgage term (years)', 1, 50, 30)

def calculate_equity_over_time(total_cost, down_payment_percent, mortgage_interest_rate, mortgage_term_years):
    down_payment = total_cost * down_payment_percent / 100
    loan_amount = total_cost - down_payment
    monthly_payment = (loan_amount * mortgage_interest_rate / 12) / (1 - (1 + mortgage_interest_rate / 12) ** (-mortgage_term_years * 12))

    equity = []
    months = []
    total_paid = 0
    remaining_balance = loan_amount

    while remaining_balance > 0:
        months.append(len(equity))
        equity_percent = (total_cost - remaining_balance) / total_cost * 100
        equity.append(equity_percent)

        interest_payment = remaining_balance * mortgage_interest_rate / 12
        principal_payment = monthly_payment - interest_payment
        total_paid += monthly_payment
        remaining_balance -= principal_payment

    fig, ax = plt.subplots()
    ax.plot(months, equity)
    ax.set_xlabel('Months')
    ax.set_ylabel('Equity Percentage')

    # Add markers for 25%, 50%, and 75% equity
    ax.axhline(y=25, color='gray', linestyle='--')
    ax.axhline(y=50, color='gray', linestyle='--')
    ax.axhline(y=75, color='gray', linestyle='--')

    return fig, equity, monthly_payment

fig, equity, monthly_payment = calculate_equity_over_time(total_cost, down_payment_percent/100, mortgage_interest_rate/100, mortgage_term_years)

st.pyplot(fig)

today = date.today()

st.write(f"Your monthly mortgage cost will be ${monthly_payment:.2f}.")

for i, eq in enumerate(equity):
    if eq >= 25:
        dt = today + timedelta(days=(i * 30))
        years = int((dt - today).days / 365.25)
        st.write(f"You will have 25% equity on {dt.strftime('%Y-%m-%d')} ({years} years from today).")
        break
else:
    st.write('You will not have 25% equity.')

for i, eq in enumerate(equity):
    if eq >= 50:
        dt = today + timedelta(days=(i * 30))
        years = int((dt - today).days / 365.25)
        st.write(f"You will have 50% equity on {dt.strftime('%Y-%m-%d')} ({years} years from today).")
        break
else:
    st.write('You will not have 50% equity.')

for i, eq in enumerate(equity):
    if eq >= 75:
        dt = today + timedelta(days=(i * 30))
        years = int((dt - today).days / 365.25)
        st.write